# Homework 5 - Visit the Wikipedia hyperlinks graph!

In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to. The ultimate goal is to rank the articles according to some criteria.

For this purpose we use the Wikipedia graph released by the SNAP group.

## Data
First of all, we upload the data from the graph and the categories.

### 1.  Articles graph

This database is a reduced version of a SNAP graph. In the database, every row is an edge and the two elements are the nodes (source and destination).

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
# Import database
articles_graph = pd.read_csv("wiki-topcats-reduced.txt", sep = "\t", header = None, names = ["Source", "Destination"])
# Convert everithing to string
articles_graph = articles_graph.applymap(str)
articles_graph.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In order to understand better the graph, we compute the edges, as a pair of tuples, of every pair of Source - Destination

In [3]:
# Edges of the graph
edges = list(zip(articles_graph.Source, articles_graph.Destination))
edges[:10]

[('52', '401135'),
 ('52', '1069112'),
 ('52', '1163551'),
 ('62', '12162'),
 ('62', '167659'),
 ('62', '279122'),
 ('62', '1089199'),
 ('62', '1354553'),
 ('62', '1400636'),
 ('62', '1403619')]

We also computes the unique nodes, as the set of all the unique nodes in the graph

In [4]:
# Compute the number of unique nodes
nodes = set(articles_graph["Source"]).union(set(articles_graph["Destination"]))
# Number of unique nodes
len(nodes)

461193

### 2. Categories

In this file, we found the information of which articles are in which of the top categories.

In [5]:
# Import database
categories = pd.read_csv("wiki-topcats-categories.txt", sep = ";", header = None, names = ['Category', 'Documents'] )
categories.head()

,Category,Documents
0,Category:Buprestoidea,301 302 303 304 305 306 307 308 309 310 311 3...
1,Category:People_from_Worcester,1056 1057 1058 1059 1060 60971 76515 76871 78...
2,Category:Skin_conditions_resulting_from_physic...,971 973 1166 1167 1168 1169 1170 1171 1172 11...
3,Category:Visual_kei_bands,1297 1300 1311 1312 1313 1314 1315 1316 1319 ...
4,Category:Japanese_rock_music_groups,1297 1300 1313 1314 1315 1316 1319 1320 1322 ...


In [6]:
# CLEANING THE DATA
# Remove the word "category"
categories['Category'] = categories['Category'].str.replace('Category:','')
# Remove "_" 
categories['Category'] = categories['Category'].str.replace('_',' ')
# The data inside "Documents" is a big string: split it in to a list
categories['Documents'] = categories['Documents'].str.split()
# From string to integer
for i in range(len(categories)):
    categories["Documents"][i] = list(map(int, categories["Documents"][i]))
categories.head()

,Category,Documents
0,Buprestoidea,"[301, 302, 303, 304, 305, 306, 307, 308, 309, ..."
1,People from Worcester,"[1056, 1057, 1058, 1059, 1060, 60971, 76515, 7..."
2,Skin conditions resulting from physical factors,"[971, 973, 1166, 1167, 1168, 1169, 1170, 1171,..."
3,Visual kei bands,"[1297, 1300, 1311, 1312, 1313, 1314, 1315, 131..."
4,Japanese rock music groups,"[1297, 1300, 1313, 1314, 1315, 1316, 1319, 132..."


We select the categories that have more than 3.500 articles. For achieve that aim, we computed the length of each list of articles, and keep only the ones that have more than 3.500 articles.

In [7]:
# Compute the length of each category 
length = []
for i in range(len(categories)):
    l = len(categories["Documents"][i])
    length.append(l)
    
# Create a new column in our df to store the length of each category
categories['Length'] = length

# Selecting only the rows that we care about
categories = categories[categories["Length"] > 3500]

# Keeping only the columns that we need
columns_of_interest = ["Category", "Documents"]
categories = categories[columns_of_interest]

# Reset the index
categories = categories.reset_index(drop = True)

# From integer to string
for i in range(len(categories)):
    categories["Documents"][i] = list(map(str, categories["Documents"][i]))
categories.head()

,Category,Documents
0,English footballers,"[22860, 28411, 28961, 28979, 29264, 29573, 295..."
1,The Football League players,"[14003, 23536, 27109, 27348, 27459, 27989, 280..."
2,Association football forwards,"[26876, 26877, 26879, 26887, 26892, 26904, 269..."
3,Association football goalkeepers,"[26900, 26909, 26917, 26960, 26966, 26984, 270..."
4,Association football midfielders,"[14003, 15291, 23536, 26880, 26882, 26885, 268..."


In order to do a better analysis, we filtered from the categories file the documents that are not in the graph.

In [11]:
# Keep only the intersection between unique_docs and the list of documents
intersection = []
for i in range(len(categories)):
    inter = set(categories["Documents"][i]).intersection(nodes)
    intersection.append(inter)
categories['Documents'] = intersection
# Make every element of "Documents", a list (from a set)
for i in range(len(categories)):
    categories["Documents"][i] = list(categories["Documents"][i])
categories

,Category,Documents
0,English footballers,"[80132, 85643, 1380888, 80947, 77702, 81156, 1..."
1,The Football League players,"[86822, 80643, 85643, 96710, 106231, 80947, 79..."
2,Association football forwards,"[78069, 1643214, 75992, 89811, 90010, 1642740,..."
3,Association football goalkeepers,"[1457698, 80048, 80643, 896622, 78701, 74424, ..."
4,Association football midfielders,"[97852, 80132, 85643, 1358075, 370343, 81156, ..."
5,Association football defenders,"[738698, 89628, 736071, 93557, 1382633, 135908..."
6,Living people,"[190325, 540645, 1503411, 379865, 666368, 4243..."
7,Year of birth unknown,"[360993, 1766474, 301834, 1378960, 47017, 6170..."
8,Harvard University alumni,"[56373, 891082, 1028988, 403757, 354252, 82274..."
9,Major League Baseball pitchers,"[1520439, 378961, 230526, 376787, 378490, 3909..."


Finally, we create a dictionary, where the key, corresponds to the name of the category, and the value, is a list with all the documents, that belongs to that category, and also belongs to the graph.

In [16]:
# Create a dictionary with categories selected in the previous step
categories_dict = dict(zip(categories.Category, categories.Documents))

# Info about our 35 surviving categories (name of the category and number of documents inside it)
i = 0
for key in categories_dict.keys():
    print(i, key, len(categories_dict[key]))
    i += 1

0 English footballers 7538
1 The Football League players 7814
2 Association football forwards 5097
3 Association football goalkeepers 3737
4 Association football midfielders 5827
5 Association football defenders 4588
6 Living people 348300
7 Year of birth unknown 2536
8 Harvard University alumni 5549
9 Major League Baseball pitchers 5192
10 Members of the United Kingdom Parliament for English constituencies 6491
11 Indian films 5568
12 Year of death missing 4122
13 English cricketers 3275
14 Year of birth missing (living people) 28498
15 Rivers of Romania 7729
16 Main Belt asteroids 11660
17 Asteroids named for people 4895
18 English-language albums 4760
19 English television actors 3362
20 British films 4422
21 English-language films 22463
22 American films 15159
23 Fellows of the Royal Society 3446
24 People from New York City 4614
25 American Jews 3411
26 American television actors 11531
27 American film actors 13865
28 Debut albums 7561
29 Black-and-white films 10759
30 Year of bir

## [RQ1] 
###  Build the graph $G=(V, E)$, where $V$ is the set of articles and $E$ the hyperlinks among them, and provide its basic information:

From the edges previously created, we created a graph dictionary, where the key is a node, and the values is a list with all the nodes, that depends on it.  

In [23]:
graph = defaultdict(list)
for i in range(len(edges)):
    key = edges[i][0]
    value = edges[i][1]
    graph[key].append(value)

In [21]:
print(dict(list(graph.items())[0:5]))

{'52': ['401135', '1069112', '1163551'], '62': ['12162', '167659', '279122', '1089199', '1354553', '1400636', '1403619', '1537692', '1544420'], '64': ['64873'], '66': ['279122', '1163290'], '74': ['279122']}


#### 1. Direct graph
Yes, the graph is direct since a hyperlink is in an article (so the edge starts from the node that represents the article) and leads to another article (the edge reaches the node representing the article to which the hyperlink refers ) and not vice versa.


#### 2. Number of nodes

In [29]:
N = len(nodes)
N

461193

####  3. Number of edges

In [30]:
E = len(edges)
E

2645247

#### 4.  Average node degree

The average degree, in the case of directed graphs is defined by: 
${\displaystyle \langle k\rangle ={\tfrac {E}{N}}}$. Were $E$ is the number of edges, and $N$ de number of nodes.

In [31]:
k = E/N
round(k,2)

5.74

#### 5. Density

For directed graphs, the graph density is defined as:

${\displaystyle D={\frac {E}{N\,(N-1)}}}$

Where $E$ is the number of edges and $N$ is the number of nodes in the graph. 

A dense graph is a graph which the number of edges is close to the maximal number of edges. The opposite, a graph with only a few edges, is a sparse graph. 

In [32]:
D = E/(N*(N-1))
D

1.2436602635647606e-05

As we can see, this is a sparse graph since the density measurement is close to zero.

## [RQ2] 
### Given a category $C_0 = \{article_1, article_2, \dots \}$ as input we want to rank all of the nodes in $N$.

### Block-ranking

We define de block ranking as:
$ block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}$

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

$distance(C_0, C_i) = median(ShortestPath(C_0, C_i))$

The lower is the distance from $C_0$, the higher is the $C_i$ position in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$ and $C_i$. 

Moreover, the length of a path is given by the sum of the weights of the edges it is composed by.

In [38]:
import ShortestPath
import numpy as np
from numpy import nanmedian
import operator

Using the shortest path functions, define in details the file "ShortestPath_How it works", and the ScoreCategories function, we are ready to compute our Block Ranking.

The ScoreCategories function, takes two categories $C$ and $D$ and computes the score, defined by the median of the set of all shortests paths between $C$ and $D$.

In [39]:
def ScoreCategories(C, D):
    cat_0 = categories_dict[C]
    cat_1 = categories_dict[D] 
    result = []
    for article_0 in cat_0:
        for article_1 in cat_1:
            sp = ShortestPath(article_0, article_1, graph)
            if sp is not None:
                result.append(sp)
    return np.median(result)

In [44]:
# Input categorie
C0 = input()

English television actors


In [46]:
ranking_dict = {}
for key in categories_dict.keys():
    if key != C0:
        score = ScoreCategories(C0, key)
        ranking_dict[key] = score
        print("score:", C0, key, "done")

score: English television actors English footballers done
score: English television actors The Football League players done
score: English television actors Association football forwards done
score: English television actors Association football goalkeepers done
score: English television actors Association football midfielders done
score: English television actors Association football defenders done
score: English television actors Living people done
score: English television actors Year of birth unknown done
score: English television actors Harvard University alumni done
score: English television actors Major League Baseball pitchers done
score: English television actors Members of the United Kingdom Parliament for English constituencies done
score: English television actors Indian films done
score: English television actors Year of death missing done
score: English television actors English cricketers done
score: English television actors Year of birth missing (living people) done


C:\Users\falli\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\falli\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


score: English television actors Rivers of Romania done
score: English television actors Main Belt asteroids done
score: English television actors Asteroids named for people done
score: English television actors English-language albums done
score: English television actors British films done
score: English television actors English-language films done
score: English television actors American films done
score: English television actors Fellows of the Royal Society done
score: English television actors People from New York City done
score: English television actors American Jews done
score: English television actors American television actors done
score: English television actors American film actors done
score: English television actors Debut albums done
score: English television actors Black-and-white films done
score: English television actors Year of birth missing done
score: English television actors Place of birth missing (living people) done
score: English television actors Artic

We have created, the ranking_dict, that is a dictionary, that has as keys all the categories but C0, and as values the score between the categories and C0. Note that, in this case the categories that has no path possible between the categorie and C0, the score is None, that is not define, that's why we change it with a greter number, in order to be last in the ranking, because is the most far away from C0 possible.  

In [55]:
for key in ranking_dict.keys():
    if ranking_dict[key] != 1.0:
        ranking_dict[key] = 999

In [56]:
ranking_dict

{'English footballers': 1.0,
 'The Football League players': 1.0,
 'Association football forwards': 1.0,
 'Association football goalkeepers': 1.0,
 'Association football midfielders': 1.0,
 'Association football defenders': 1.0,
 'Living people': 1.0,
 'Year of birth unknown': 1.0,
 'Harvard University alumni': 1.0,
 'Major League Baseball pitchers': 1.0,
 'Members of the United Kingdom Parliament for English constituencies': 1.0,
 'Indian films': 1.0,
 'Year of death missing': 1.0,
 'English cricketers': 1.0,
 'Year of birth missing (living people)': 1.0,
 'Rivers of Romania': 999,
 'Main Belt asteroids': 1.0,
 'Asteroids named for people': 1.0,
 'English-language albums': 1.0,
 'British films': 1.0,
 'English-language films': 1.0,
 'American films': 1.0,
 'Fellows of the Royal Society': 1.0,
 'People from New York City': 1.0,
 'American Jews': 1.0,
 'American television actors': 1.0,
 'American film actors': 1.0,
 'Debut albums': 1.0,
 'Black-and-white films': 1.0,
 'Year of birth mi

We define ranking, as a list that contains all the categories, sorted by score, where the input categorie is the first one.

In [57]:
# All the sorted categories, but C0
L = sorted(ranking_dict, key=ranking_dict.__getitem__)
ranking = [C0]
ranking = ranking + L
# Thw complete ranking list, including C0
ranking

['English television actors',
 'English footballers',
 'The Football League players',
 'Association football forwards',
 'Association football goalkeepers',
 'Association football midfielders',
 'Association football defenders',
 'Living people',
 'Year of birth unknown',
 'Harvard University alumni',
 'Major League Baseball pitchers',
 'Members of the United Kingdom Parliament for English constituencies',
 'Indian films',
 'Year of death missing',
 'English cricketers',
 'Year of birth missing (living people)',
 'Main Belt asteroids',
 'Asteroids named for people',
 'English-language albums',
 'British films',
 'English-language films',
 'American films',
 'Fellows of the Royal Society',
 'People from New York City',
 'American Jews',
 'American television actors',
 'American film actors',
 'Debut albums',
 'Black-and-white films',
 'Year of birth missing',
 'Place of birth missing (living people)',
 'Article Feedback Pilot',
 'American military personnel of World War II',
 'Windows g

Finally, we got the block ranking vector, so we can start to sort the nodes in each category.

We will sort them computing subgraph induced by each category, following the order of the block ranking vector. The procedure to get subgraph induced by c0 is different from that used for the other categories, because there aren't previous category to c0 in the block ranking vector. So we will compute it separately from the others.

First of all, we create a new dictionary (graph_1) within which, for each article in the "Destination" column of the dataframe categories, there is a list of all the articles in the "Source" column that reach this article. This step is essential to build subgraphs.

In [58]:
edges = list(zip(articles_graph.Destination, articles_graph.Source))

In [59]:
graph_1 = defaultdict(list)
for i in range(len(edges)):
    key = edges[i][0]
    value = edges[i][1]
    graph_1[key].append(value)

In [60]:
# Now we can compute the subgraph induced by c0. We record the score of each article of c0 in a dictionary 
W_c0=defaultdict(int)
for node in categories_dict[ranking[0]]:
    w=len(graph_1[node])
    W_c0[node]=w

In [62]:
print(dict(list(W_c0.items())[0:5]))

defaultdict(<class 'int'>, {'1044787': 74, '1034389': 7, '1080500': 1, '1039577': 28, '1039555': 152, '719743': 0, '1042587': 16, '1040705': 1, '1180951': 61, '1037871': 3, '1789734': 1, '1033451': 0, '1339329': 0, '1034249': 8, '1035363': 0, '724793': 12, '1061561': 113, '1470395': 1, '1061003': 62, '1035859': 2, '255982': 6, '1035088': 5, '1040055': 2, '1059228': 52, '1034044': 35, '1064458': 27, '1036255': 39, '1040397': 3, '1033666': 5, '1034477': 26, '1165447': 24, '376202': 1, '1028673': 6, '1040328': 3, '1316598': 0, '1042702': 30, '1040359': 15, '1035259': 1, '1038366': 24, '1355914': 1, '1045397': 10, '1039440': 7, '631984': 5, '1033505': 5, '1046374': 22, '1039414': 16, '1063689': 26, '722720': 5, '739530': 6, '604272': 0, '719512': 2, '1040307': 10, '1061449': 317, '1031682': 2, '1033956': 6, '1031754': 6, '1033941': 3, '1043877': 6, '1038279': 12, '430782': 1, '1035182': 4, '1037657': 2, '1039973': 25, '1043169': 37, '1032713': 1, '1035448': 0, '1040190': 4, '1033579': 2, '

Now we can extend the graph to nodes of the other categories, keeping in mind that the in-edges coming from the previous category have as weights the score of the node that sends the edge. Every extension of the graph will be stored in a list (Subgraph induced by c0 already stored) 

In [63]:
subgraph=[]
subgraph.append(W_c0)

In [64]:
for i in range(1,len(ranking)):
    W_c=defaultdict(int)
    for node in categories_dict[ranking[i]]:
        w_c=0
        if node not in subgraph[i-1]:
            for inedge in graph_1[node]:
                if inedge in subgraph[i-1]:
                    w_c+=subgraph[i-1][inedge]
                else:
                    w_c+=1
            W_c[node]=w_c
    subgraph.append(W_c)

In [66]:
print(dict(list(W_c.items())[0:5]))

defaultdict(<class 'int'>, {'788128': 1, '788316': 1, '784722': 1, '1351067': 2, '938634': 1, '784840': 2, '784703': 1, '722089': 1, '786542': 1, '783809': 1, '786134': 1, '1622639': 1, '217543': 2, '784652': 1, '785116': 1, '217724': 1, '1633370': 1, '785311': 1, '787529': 1, '785611': 1, '786089': 1, '784919': 1, '787539': 1, '217513': 1, '785201': 1, '784034': 1, '1000397': 1, '786297': 1, '1622697': 1, '217931': 1, '787836': 1, '938389': 1, '722047': 1, '217602': 1, '1622342': 1, '1658658': 2, '996123': 1, '787546': 1, '721934': 1, '1622371': 1, '785438': 1, '786623': 9, '783836': 1, '999696': 1, '217735': 2, '1622447': 0, '784903': 7, '787709': 1, '722549': 1, '787508': 4, '940775': 1, '784190': 1, '1687980': 1, '1622522': 1, '927662': 1, '786629': 1, '784067': 1, '1480219': 1, '1622321': 1, '722094': 1, '1622182': 1, '996341': 1, '722182': 1, '786559': 1, '1686955': 1, '785330': 1, '788307': 1, '722539': 1, '787017': 1, '217581': 9, '787879': 1, '784479': 1, '995060': 1, '1622357

Now we can extend the graph to nodes of the other categories, keeping in mind that the in-edges coming from the previous category have as weights the score of the node that sends the edge. Every extension of the graph will be stored in a list (Subgraph induced by c0 already stored) 

In [63]:
subgraph=[]
subgraph.append(W_c0)

In [64]:
for i in range(1,len(ranking)):
    W_c=defaultdict(int)
    for node in categories_dict[ranking[i]]:
        w_c=0
        if node not in subgraph[i-1]:
            for inedge in graph_1[node]:
                if inedge in subgraph[i-1]:
                    w_c+=subgraph[i-1][inedge]
                else:
                    w_c+=1
            W_c[node]=w_c
    subgraph.append(W_c)

In [66]:
print(dict(list(W_c.items())[0:5]))

defaultdict(<class 'int'>, {'788128': 1, '788316': 1, '784722': 1, '1351067': 2, '938634': 1, '784840': 2, '784703': 1, '722089': 1, '786542': 1, '783809': 1, '786134': 1, '1622639': 1, '217543': 2, '784652': 1, '785116': 1, '217724': 1, '1633370': 1, '785311': 1, '787529': 1, '785611': 1, '786089': 1, '784919': 1, '787539': 1, '217513': 1, '785201': 1, '784034': 1, '1000397': 1, '786297': 1, '1622697': 1, '217931': 1, '787836': 1, '938389': 1, '722047': 1, '217602': 1, '1622342': 1, '1658658': 2, '996123': 1, '787546': 1, '721934': 1, '1622371': 1, '785438': 1, '786623': 9, '783836': 1, '999696': 1, '217735': 2, '1622447': 0, '784903': 7, '787709': 1, '722549': 1, '787508': 4, '940775': 1, '784190': 1, '1687980': 1, '1622522': 1, '927662': 1, '786629': 1, '784067': 1, '1480219': 1, '1622321': 1, '722094': 1, '1622182': 1, '996341': 1, '722182': 1, '786559': 1, '1686955': 1, '785330': 1, '788307': 1, '722539': 1, '787017': 1, '217581': 9, '787879': 1, '784479': 1, '995060': 1, '1622357